In [45]:
import sys
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [46]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

mA = mss.add (Mass(1, (1,0,0)))
mB = mss.add (Mass(2, (2,0,0)))
f1 = mss.add (Fix( (0,0,0)) )
mss.add (Spring(1, 200000, (f1, mA)))
mss.add (Spring(1, 100000, (mA, mB)))

1

In [47]:
# Adding a chain as small stiff spring elements.
N = 10
segment_length = 0.1
mass_value = 0.1
prev_pos = (2.0, 0.0, 0.0)
prev_connector = mB

for i in range(N):
    new_pos = (prev_pos[0] + segment_length, prev_pos[1], prev_pos[2])
    new_mass = mss.add(Mass(mass_value, new_pos))
    mss.add(Spring(0.1, 1000.0, (prev_connector, new_mass)))
    prev_pos = new_pos
    prev_connector = new_mass


In [ ]:
masses = []
for m in mss.masses:
    masses.append(Mesh(SphereBufferGeometry(0.2, 16, 16), MeshStandardMaterial(color='red'), position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(Mesh(SphereBufferGeometry(0.2, 32, 16), MeshStandardMaterial(color='blue'), position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeom = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan') # only visible in longr chain elements
springs = LineSegments2(springgeom, m2)    

axes = AxesHelper(1)

In [ ]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height, up=[0,0,1])
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [50]:
from time import sleep
for i in range(10000):
    mss.simulate (0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)